In [6]:
try:
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import seaborn as sns
    import tensorflow as tf
    import sys
    import os

    # Add the main directory (where common.py is located) to the Python path
    sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
    from common_functions import data_validation_filtering,kWh_validation
    from tensorflow import keras
    from tensorflow.keras import layers
    print("tensorflow version:",tf.__version__)
    np.set_printoptions(precision=3, suppress=True)
except Exception as e:
    print("error in importing :",e)

tensorflow version: 2.17.0


In [2]:
raw_dataset = pd.read_csv(r"C:\Users\HARSH\Downloads/surbhi.csv")
df = raw_dataset.copy()

In [3]:
print("null:",df.isna().sum().sum())
print("shape:",df.shape)
df.info()

null: 0
shape: (5000, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0    FlatID    5000 non-null   float64
 1   kWh        5000 non-null   int64  
 2   R_Voltage  5000 non-null   float64
 3   Y_Voltage  5000 non-null   float64
 4   B_Voltage  5000 non-null   float64
 5   R_Current  5000 non-null   float64
 6   Y_Current  5000 non-null   float64
 7   B_Current  5000 non-null   float64
 8   R_PF       5000 non-null   float64
 9   Y_PF       5000 non-null   float64
 10  B_PF       5000 non-null   float64
 11  Frequency  5000 non-null   float64
 12  Load_kW    5000 non-null   float64
 13  Load_kVA   5000 non-null   float64
 14  Date_Time  5000 non-null   object 
dtypes: float64(13), int64(1), object(1)
memory usage: 586.1+ KB


In [4]:
df.drop(" FlatID",axis= 1,inplace= True)
df['Date_Time'] = pd.to_datetime(df['Date_Time'],dayfirst=True)
df.set_index("Date_Time", drop= True, inplace= True)
df.sort_index(inplace=True)
df.head()

,kWh,R_Voltage,Y_Voltage,B_Voltage,R_Current,Y_Current,B_Current,R_PF,Y_PF,B_PF,Frequency,Load_kW,Load_kVA
Date_Time,,,,,,,,,,,,,
2024-06-11 19:52:00,2150,232.4,230.9,230.7,0.08,0.58,0.06,1.0,0.82,1.00,50.00,0.13,0.15
2024-06-11 19:58:00,2150,232.6,230.8,230.8,0.08,0.58,0.06,1.0,0.81,0.92,49.96,0.13,0.15
2024-06-11 20:03:00,2150,234.0,231.8,232.1,0.08,0.58,0.06,1.0,0.81,0.92,50.04,0.13,0.15
2024-06-11 20:09:00,2150,233.4,231.8,231.8,0.08,0.58,0.06,1.0,0.81,0.92,50.03,0.13,0.15
2024-06-11 20:14:00,2150,233.5,231.7,231.9,0.09,0.58,0.06,1.0,0.81,0.92,49.98,0.13,0.15


In [5]:
clean_df = data_validation_filtering(df.copy())
final_df = kWh_validation(clean_df)
# print("null:",final_df.isna().sum().sum())
final_df

NameError: name 'data_validation_filtering' is not defined

In [ ]:
column_indices = {name: i for i, name in enumerate(final_df.columns)}
column_indices
n = len(final_df)
train_df = final_df[0:int(n*0.7)]
val_df = final_df[int(n*0.7):int(n*0.9)]
test_df = final_df[int(n*0.9):]

num_features = final_df.shape[1]

In [ ]:
train_df

In [ ]:
val_df

In [ ]:
test_df

In [ ]:
train_df.describe().transpose()

In [ ]:
train_features = train_df.copy()
test_features = test_df.copy()

train_labels = train_features.pop('consumed_unit')
test_labels = test_features.pop('consumed_unit')

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)

In [ ]:
normalizer.adapt(np.array(train_features))

In [ ]:
print(normalizer.mean.numpy())

In [ ]:
first = np.array(train_features[:10])

with np.printoptions(precision=2, suppress=True):
  print('First example:\n', first)
  print()
  print('Normalized:\n', normalizer(first).numpy())

In [ ]:
forecasting = np.array(train_features)

forecasting_normalizer = layers.Normalization(axis=None)
forecasting_normalizer.adapt(forecasting)

In [ ]:
forecasting_model = tf.keras.Sequential([
    forecasting_normalizer,
    layers.Dense(units=1)
])

forecasting_model.summary()

In [ ]:
forecasting_model.predict(train_features[:10])

In [ ]:
forecasting_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:
%%time
history = forecasting_model.fit(
    train_features,
    train_labels,
    epochs=100,
    # Suppress logging.
    verbose=0,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.ylim([0, 10])
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plt.legend()
    plt.grid(True)
plot_loss(history)

In [ ]:
test_results = {}

test_results['forecasting_model'] = forecasting_model.evaluate(
    test_features,
    test_labels, verbose=0)

In [ ]:
x = tf.linspace(0.0, 250, 251)
y = forecasting_model.predict(x)

In [ ]:
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

In [ ]:
linear_model.predict(train_features[:10])

In [ ]:
linear_model.layers[1].kernel

In [ ]:
linear_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:
%%time
history = linear_model.fit(
    train_features,
    train_labels,
    epochs=100,
    # Suppress logging.
    verbose=0,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

In [ ]:
test_results['linear_model'] = linear_model.evaluate(
test_features, test_labels, verbose=0)

In [ ]:
test_results